In [71]:
import os
print(os.getcwd())
import sys
sys.path.append('/scratch/ty296/CT_MPS_mini')
import h5py
import numpy as np
from read_hdf5_func import von_neumann_entropy_sv

/scratch/ty296


In [72]:
import pandas as pd
import glob
import os
import h5py

dir_name = '/scratch/ty296/hdf5_data/p_ctrl0.4'
h5_files = glob.glob(os.path.join(dir_name, '*.h5'))
filenum = 1
file_name = os.path.join(dir_name, list(os.listdir(dir_name))[filenum])

with h5py.File(file_name, 'r') as f:
    # f.visititems(print)
    # print(f.keys())
    # print(f['metadata'])
    # print(f['singular_values'])
    # print(dir(f))
    metadata = f['metadata']
    print(f.keys())
    print(metadata.keys())
    singular_values = f['singular_values']
    print(singular_values.keys())
    # print(metadata['result_1']['args']['L'][()])
    # singular_values = f['singular_values']
    # print(singular_values['result_1'][()])


<KeysViewHDF5 ['metadata', 'singular_values']>
<KeysViewHDF5 ['result_1', 'result_10', 'result_100', 'result_101', 'result_102', 'result_103', 'result_104', 'result_105', 'result_106', 'result_107', 'result_108', 'result_109', 'result_11', 'result_110', 'result_111', 'result_112', 'result_113', 'result_114', 'result_115', 'result_116', 'result_117', 'result_118', 'result_119', 'result_12', 'result_120', 'result_121', 'result_122', 'result_123', 'result_124', 'result_125', 'result_126', 'result_127', 'result_128', 'result_129', 'result_13', 'result_130', 'result_131', 'result_132', 'result_133', 'result_134', 'result_135', 'result_136', 'result_137', 'result_138', 'result_139', 'result_14', 'result_140', 'result_141', 'result_142', 'result_143', 'result_144', 'result_145', 'result_146', 'result_147', 'result_148', 'result_149', 'result_15', 'result_150', 'result_151', 'result_152', 'result_153', 'result_154', 'result_155', 'result_156', 'result_157', 'result_158', 'result_159', 'result_

In [73]:
print(h5_files)

['/scratch/ty296/hdf5_data/p_ctrl0.4/46152168_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152141_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46210854_a0_L24.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46210840_a0_L24.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46151328_a0_L12.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46151375_a0_L12.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152089_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152290_a0_L16.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46151335_a0_L12.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152114_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152045_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46151322_a0_L12.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46210927_a0_L24.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46210892_a0_L24.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152076_a0_L20.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46152378_a0_L16.h5', '/scratch/ty296/hdf5_data/p_ctrl0.4/46151266_a0_L12.h5', '/scratch/ty296/hdf5_data/p_ct

In [87]:
import h5py
import glob
import os
import tqdm
def postprocessing(sv_combined="/scratch/ty296/hdf5_data_combined/sv_combined.h5", dir_name='/scratch/ty296/hdf5_data/p_ctrl0.4'):
    h5_files = glob.glob(os.path.join(dir_name, '*.h5'))
    with h5py.File(sv_combined, 'w') as f_combined:
        counter = 0
        for file in tqdm.tqdm(h5_files):
            with h5py.File(file, 'r') as f:
                metadata = f['metadata']
                singular_values = f['singular_values']
                for result_group in metadata.keys():
                    counter += 1
                    p_proj = metadata[result_group]['p_proj'][()]
                    p_ctrl = metadata[result_group]['p_ctrl'][()]
                    L = metadata[result_group]['args']['L'][()]
                    maxbond = metadata[result_group]['max_bond'][()]
                    sv_arr = singular_values[result_group][()]
                    group_name = f'real{counter}'
                    grp = f_combined.create_dataset(group_name, data=sv_arr)
                    grp.attrs['p_proj'] = p_proj
                    grp.attrs['p_ctrl'] = p_ctrl
                    grp.attrs['L'] = L
                    grp.attrs['maxbond'] = maxbond

In [ ]:
from typing import Dict, List, Tuple, Optional

def calculate_mean_and_error(ee_values: List[float]) -> Tuple[float, float]:
    """Calculate mean and standard error of the mean."""
    ee_array = np.array(ee_values)
    mean = np.mean(ee_array)
    sem = np.std(ee_array, ddof=1) / np.sqrt(len(ee_array))
    return mean, sem

def calculate_variance_and_error(ee_values: List[float]) -> Tuple[float, float]:
    """Calculate variance and standard error of variance."""
    ee_array = np.array(ee_values)
    variance = np.var(ee_array, ddof=1)
    # Standard error of variance approximation
    n = len(ee_array)
    se_var = variance * np.sqrt(2.0 / (n - 1))
    return variance, se_var

NameError: name 'List' is not defined

In [ ]:
hdf5_combined = "/scratch/ty296/hdf5_data_combined/sv_combined.h5"

with h5py.File(hdf5_combined, 'r') as f:
    from collections import defaultdict
    groups = defaultdict(list)
    for real_key in f.keys():
        s0 = von_neumann_entropy_sv(f[real_key][()], n=0, positivedefinite=False, threshold=1e-1)
        # print(f[real_key].attrs['p_proj'],f[real_key].attrs['p_ctrl'],f[real_key].attrs['L'],f[real_key].attrs['maxbond'],s0)
        key_val = (f[real_key].attrs['L'],f[real_key].attrs['p_ctrl'],f[real_key].attrs['p_proj'])
        groups[key_val].append(s0)
    
    for key_val, s0_list in groups.items():
        print(key_val, "mean", np.mean(s0_list), "sem", np.std(s0_list))


(np.int64(20), np.float64(0.4), np.float64(0.5)) [np.float64(1.0986122886681098), np.float64(1.0986122886681098), np.float64(1.3862943611198906), np.float64(1.9459101490553132), np.float64(2.0794415416798357), np.float64(1.0986122886681098), np.float64(0.6931471805599453), np.float64(1.6094379124341003), np.float64(1.9459101490553132), np.float64(1.9459101490553132), np.float64(2.5649493574615367), np.float64(2.0794415416798357), np.float64(1.6094379124341003), np.float64(1.0986122886681098), np.float64(2.4849066497880004), np.float64(1.6094379124341003), np.float64(2.302585092994046), np.float64(1.9459101490553132), np.float64(0.6931471805599453), np.float64(1.6094379124341003), np.float64(1.0986122886681098), np.float64(2.1972245773362196), np.float64(1.0986122886681098), np.float64(1.3862943611198906), np.float64(2.1972245773362196), np.float64(1.0986122886681098), np.float64(2.5649493574615367), np.float64(1.791759469228055), np.float64(1.6094379124341003), np.float64(2.63905732961

In [102]:
sys.getsizeof(groups)

2280